# Sentiment Analysis with Hugging Face

Hugging Face is an open-source and platform provider of machine learning technologies. You can use install their package to access some interesting pre-built models to use them directly or to fine-tune (retrain it on your dataset leveraging the prior knowledge coming with the first training), then host your trained models on the platform, so that you may use them later on other devices and apps.

Please, [go to the website and sign-in](https://huggingface.co/) to access all the features of the platform.

[Read more about Text classification with Hugging Face](https://huggingface.co/tasks/text-classification)

The Hugging face models are Deep Learning based, so will need a lot of computational GPU power to train them. Please use [Colab](https://colab.research.google.com/) to do it, or your other GPU cloud provider, or a local machine having NVIDIA GPU.

In [1]:
!pip install huggingface_hub transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.2 MB/s eta 0:00:00


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
# Import libraries
import os
import uuid
import pandas as pd
import numpy as np

from google.colab import drive
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    IntervalStrategy,
    TrainingArguments,
    EarlyStoppingCallback,
    Trainer
) 

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


## Application of Hugging Face Text classification model Fune-tuning

Find below a simple example, with just `3 epochs of fine-tuning`. 

Read more about the fine-tuning concept : [here](https://deeplizard.com/learn/video/5T-iXNNiwIs#:~:text=Fine%2Dtuning%20is%20a%20way,perform%20a%20second%20similar%20task.)

The datasets package is a Python library that provides a collection of over 100 natural language processing (NLP) datasets commonly used for research and development. The library is designed to provide easy access to these datasets, as well as a uniform interface for loading, preprocessing, and working with the data.

The datasets include a range of tasks such as text classification, question answering, named entity recognition, and sentiment analysis, and cover a variety of languages including English, Spanish, French, Chinese, and many others. Some of the popular datasets included in the package are IMDB, COCO, SQuAD, Multi30k, Wikipedia, and Amazon Reviews.

The datasets package is developed by Hugging Face, a company that specializes in NLP and provides a suite of libraries and tools for working with NLP models.




This code sets the environment variable "WANDB_DISABLED" to "true", which disables the use of the Weights and Biases (W&B) tool. W&B is a third-party tool that can be used to track and visualize the training progress of machine learning models. By setting this environment variable, you are telling your code to not use this tool.

In [6]:
# Disabe W&B
os.environ["WANDB_DISABLED"] = "true"

In [7]:
# Load the dataset and display some values

# Load the CSV file into a DataFrame

url = "https://github.com/Azubi-Africa/Career_Accelerator_P5-NLP/raw/master/zindi_challenge/data/Train.csv"

df = pd.read_csv(url)


Data Quality checks 

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


In [9]:
# Select rows with missing values
df.isnull().sum()

tweet_id     0
safe_text    0
label        1
agreement    2
dtype: int64

In [10]:
# Select rows with missing values
df[df.isnull().any(axis=1)]

,tweet_id,safe_text,label,agreement
4798,RQMQ0L2A,#lawandorderSVU,NaN,NaN
4799,I cannot believe in this day and age some pare...,1,0.666667,NaN


In [11]:
# Extract complete text from 'safe_text' column
complete_text = df.iloc[4798]['safe_text']
complete_text

'#lawandorderSVU '

In [12]:
# Select row by index and assign values to columns
df.loc[4798, 'label'] = 0
df.loc[4798, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df.iloc[4798, df.columns.get_loc('safe_text')] = complete_text


In [13]:
# Generate random UUID string for tweet_id
'''UUIDs are often used in software applications for various purposes such as generating unique IDs for entities, 
tracking unique user sessions, or creating unique file names'''
rand_tweet_id = str(uuid.uuid4())

# Select row by index and assign values to columns
row_index = 4799
df.loc[row_index, 'tweet_id'] = rand_tweet_id
df.loc[row_index, 'label'] = 1
df.loc[row_index, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df.iloc[row_index, df.columns.get_loc('safe_text')] = df.iloc[row_index, 1]


In [14]:
df[df.duplicated()].sum()

tweet_id     0.0
safe_text    0.0
label        0.0
agreement    0.0
dtype: float64

I manually split the training set to have a training subset ( a dataset the model will learn on), and an evaluation subset ( a dataset the model with use to compute metric scores to help use to avoid some training problems like [the overfitting](https://www.ibm.com/cloud/learn/overfitting) one ). 

There are multiple ways to do split the dataset. You'll see two commented line showing you another one.

In [15]:
# Split the train data => {train, eval}
train, eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [16]:
train.head()

,tweet_id,safe_text,label,agreement
1641,CQDD6QLM,"New <user> ""Hey Love"" #MMR #ManyMenRecords #Yo...",0.0,1.000000
3907,5GV8NEZS,S1256 [NEW] Extends exemption from charitable ...,0.0,1.000000
336,I4D043ST,<user> esp when mercury free vaccines are avai...,1.0,0.666667
6861,CKX52Y8G,"My Life, Your Entertainment #YOTC #MMR @ Exoti...",0.0,1.000000
720,07S3NL2T,Baby Luna is sore from her vaccines :( #poorpuppy,0.0,0.666667


In [17]:
eval.head()

,tweet_id,safe_text,label,agreement
5818,Y8PQ0BT7,So nervous... The baby's getting vaccines... (...,1.0,0.666667
7842,C9Z6JBSS,AIDS N : A malaria vaccine in children with HI...,0.0,0.666667
880,0VE4NWWQ,Measles Outbreak Hits Texas Church That Preach...,1.0,0.666667
9072,RHQRUF14,Thank you <user> for mtg with your staff. We l...,1.0,1.000000
288,ZWEP2IL4,Health district offers no-cost immunizations f...,1.0,0.666667


In [18]:
print(f"new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

new dataframe shapes: train is (8000, 4), eval is (2001, 4)


By saving the subsets as CSV files, you can easily load them into your machine learning framework of choice (e.g., PyTorch, TensorFlow) and preprocess the data as needed for your specific task. Additionally, saving the subsets as separate files allows you to easily swap in new training or evaluation data as needed during the development process.

In [19]:
# Save splitted subsets

# Define file path

file_path = '/content/drive/MyDrive/NLP-Sentiment-Classification'

#"/content/drive/MyDrive/NLP-Sentiment-Classification "

train.to_csv(os.path.join(file_path, "train_subset.csv"), index=False)
eval.to_csv(os.path.join(file_path, "eval_subset.csv"), index=False)

In [20]:
'''
This code is loading the training and evaluation 
subsets of the dataset that were saved as CSV files in the previous 
step into a Dataset object using the load_dataset 
function provided by the datasets library
'''
# Load the CSV files into a dataset

# Next, we use the `load_dataset` function to load the training and evaluation subsets of the dataset
# that we saved as CSV files in the previous step into a `Dataset` object.
# We pass the function the following arguments:
# - The first argument tells the function to load data from CSV files.
# - The `data_files` parameter is passed a dictionary containing the paths to the CSV files
#   for the training and evaluation subsets.
# - The `encoding` parameter is set to `'ISO-8859-1'` to specify the character encoding of the CSV files.


dataset = load_dataset('csv', data_files={
    'train': '/content/drive/MyDrive/NLP-Sentiment-Classification/train_subset.csv',
    'eval': '/content/drive/MyDrive/NLP-Sentiment-Classification/eval_subset.csv'
}, encoding='ISO-8859-1')


'''
The choice of ISO-8859-1 encoding was likely based on the fact that the original text data was in English 
and that ISO-8859-1 can represent all the characters in the English language.
However, the choice of encoding scheme can depend on various factors, 
such as the source of the text data, the software used to process the data, and 
any specific requirements of the project. 
For example, if the text data includes non-English characters, 
then a different encoding scheme may be necessary to ensure that all 
characters are properly represented. Similarly, if the project involves using 
certain software libraries that only support certain encoding schemes, 
then that may influence the choice of encoding scheme.

'''

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b7363c87a55b44bd/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

'\nThe choice of ISO-8859-1 encoding was likely based on the fact that the original text data was in English \nand that ISO-8859-1 can represent all the characters in the English language.\nHowever, the choice of encoding scheme can depend on various factors, \nsuch as the source of the text data, the software used to process the data, and \nany specific requirements of the project. \nFor example, if the text data includes non-English characters, \nthen a different encoding scheme may be necessary to ensure that all \ncharacters are properly represented. Similarly, if the project involves using \ncertain software libraries that only support certain encoding schemes, \nthen that may influence the choice of encoding scheme.\n\n'

Transformers is a Python library for natural language processing (NLP) developed by Hugging Face. It provides an easy-to-use interface for building and training state-of-the-art deep learning models for a variety of NLP tasks, such as text classification, named entity recognition, question answering, and more.

The transformer architecture is a type of neural network that is particularly well-suited for processing sequential data, such as natural language text. It replaces the recurrent neural networks (RNNs) and convolutional neural networks (CNNs) that were previously used for NLP tasks, and has achieved state-of-the-art performance on a wide range of benchmarks.

The Transformers library provides pre-trained transformer models that can be fine-tuned on a specific NLP task with only a small amount of task-specific data. This allows developers to easily leverage the power of transformer models for their own NLP tasks, even if they do not have access to large amounts of training data or high-performance computing resources.

A tokenizer is a component in natural language processing (NLP) that breaks down text into individual tokens, which are usually words or subwords. Tokenization is an important preprocessing step in many NLP tasks, because it converts raw text data into a format that can be easily processed by machine learning models.

There are different types of tokenizers that can be used, depending on the specific requirements of the task. Some common types include:

Word tokenizers: These tokenize text into individual words based on whitespace or punctuation.

Subword tokenizers: These tokenize text into subwords, which can be useful for handling out-of-vocabulary words or words that are rare in the training data.

Character tokenizers: These tokenize text into individual characters, which can be useful for languages that have complex orthographies or for handling misspellings.

AutoTokenizer is used to instantiate a tokenizer. AutoTokenizer is a class in the Transformers library that provides a convenient way to automatically select the appropriate tokenizer for a given pre-trained model. The AutoTokenizer class uses heuristics to determine the type of tokenizer that should be used based on the architecture and configuration of the pre-trained model. This can be useful when working with a variety of pre-trained models, because it allows you to use the appropriate tokenizer without having to manually select one for each model.

In [21]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer_roberta = AutoTokenizer.from_pretrained('roberta-base')
tokenizer_distilbert = AutoTokenizer.from_pretrained('distilbert-base-uncased')

'''
This code instantiates a tokenizer for the BERT (Bidirectional Encoder Representations from Transformers) 
pre-trained model with the bert-base-cased configuration.

'''


'\nThis code instantiates a tokenizer for the BERT (Bidirectional Encoder Representations from Transformers) \npre-trained model with the bert-base-cased configuration.\n\n'

Specifically, AutoTokenizer.from_pretrained() is a method in the Transformers library that allows you to load a pre-trained tokenizer for a specific model architecture and configuration. In this case, the from_pretrained() method is called with the argument 'bert-base-cased', which is the name of a pre-trained BERT model that has been trained on a large corpus of English text.

The bert-base-cased configuration refers to a version of the BERT model that has a cased vocabulary, meaning that it distinguishes between uppercase and lowercase letters. This can be useful in tasks where the case of words is important, such as named entity recognition or sentiment analysis.

By instantiating a tokenizer for the bert-base-cased model using AutoTokenizer.from_pretrained(), you can tokenize text according to the same scheme used during pre-training of the BERT model. This can be useful when fine-tuning the pre-trained model on a specific task, because it ensures that the input data is pre-processed in the same way as the data used to train the original model.

In [22]:
# Define a function to transform the label values
def transform_labels(label):
    # Extract the label value
    label = label['label']
    # Map the label value to an integer value
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2
    # Return a dictionary with a single key-value pair
    return {'labels': num}

# Define a function to tokenize the text data
def tokenize_data(example):
    # Extract the 'safe_text' value from the input example and tokenize it
    return tokenizer(example['safe_text'], padding='max_length')

# Define a function to tokenize the text data
def tokenize_data2(example):
    # Extract the 'safe_text' value from the input example and tokenize it
    return tokenizer_roberta(example['safe_text'], padding='max_length')

# Define a function to tokenize the text data
def tokenize_data3(example):
    # Extract the 'safe_text' value from the input example and tokenize it
    return tokenizer_distilbert(example['safe_text'], padding='max_length')

# Apply the transformation functions to the dataset using the 'map' method
# This transforms the label values and tokenizes the text data
dataset_out = dataset.map(transform_labels)

dataset_base = dataset_out.map(tokenize_data, batched=True)

dataset_roberta = dataset_out.map(tokenize_data2, batched=True)

dataset_distilbert = dataset_out.map(tokenize_data3, batched=True)

# Define a list of column names to remove from the dataset
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']

# Apply the 'transform_labels' function to the dataset to transform the label values
# Also remove the columns specified in 'remove_columns'

dataset_base = dataset_base.map(transform_labels, remove_columns=remove_columns)
dataset_roberta = dataset_roberta.map(transform_labels, remove_columns=remove_columns)
dataset_distilbert = dataset_distilbert.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

The columns specified in remove_columns are removed from the dataset because they are not needed for the subsequent analysis or model training.

tweet_id: This column contains unique identifiers for each tweet, which are not relevant for the analysis or modeling.

label: This column contains the original label values, which have already been transformed into numerical values using the transform_labels function.

safe_text: This column contains the preprocessed text data that has already been tokenized and encoded, so it is not needed for subsequent analysis or modeling.

agreement: This column indicates the level of agreement among the annotators for each tweet. While this information might be useful for some analyses, it is not necessary for the sentiment analysis task at hand.

By removing these columns, the resulting dataset is more compact and easier to work with, while retaining all the relevant information for the sentiment analysis task.

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tweet_id', 'safe_text', 'label', 'agreement'],
        num_rows: 8000
    })
    eval: Dataset({
        features: ['tweet_id', 'safe_text', 'label', 'agreement'],
        num_rows: 2001
    })
})

In [23]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',                          # Directory where the model checkpoints and evaluation results will be stored
    evaluation_strategy=IntervalStrategy.STEPS,      # Interval for evaluating the model during training (every specified number of steps)
    save_strategy=IntervalStrategy.STEPS,            # Interval for saving the model during training (every specified number of steps)
    save_steps=500,                                  # Number of steps between two saves
    load_best_model_at_end=True,                     # Whether to load the best model at the end of training
    num_train_epochs=1,                              # Number of training epochs
    per_device_train_batch_size=2,                   # Batch size per GPU for training
    per_device_eval_batch_size=3,                    # Batch size per GPU for evaluation
    learning_rate=5e-5,                              # Learning rate
    weight_decay=0.01,                               # Weight decay
    warmup_steps=500,                                # Number of warmup steps
    logging_steps=500,                               # Number of steps between two logs
    fp16=True,                                       # Whether to use 16-bit precision
    gradient_accumulation_steps=16,                  # Number of steps to accumulate gradients before performing an optimizer step
    dataloader_num_workers=2,                        # Number of workers to use for loading data
    push_to_hub=True,                                # Whether to push the model checkpoints to the Hugging Face hub
    push_to_hub_model_id="finetuned-Sentiment-classfication-model",  # Model ID to use when pushing the model to the Hugging Face hub
)

# Define the early stopping callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,                       # Number of epochs with no improvement before stopping training
    early_stopping_threshold=0.01,                   # Minimum improvement in the metric for considering an improvement
)

# Combine the training arguments and the early stopping callback
training_args.callbacks = [early_stopping]


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.9/dist-packages/transformers/training_args.py:1469: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Abubakari/finetuned-Sentiment-classfication-model).
  warnings.warn(


Explanation:

from transformers import IntervalStrategy, TrainingArguments: Importing the IntervalStrategy and TrainingArguments classes from the transformers library.

training_args = TrainingArguments(: Creating a TrainingArguments object and assigning it to the variable training_args.

output_dir='./results': Specifies the directory where the training results will be saved.

evaluation_strategy=IntervalStrategy.STEPS: Specifies how often the model will be evaluated during training. In this case, the model will be evaluated at specific intervals.

save_strategy=IntervalStrategy.STEPS: Specifies how often the model will be saved during training. In this case, the model will be saved at specific intervals.

save_steps=500: Specifies how often the model will be saved during training, in terms of the number of steps taken. In this case, the model will be saved every 500 steps.

load_best_model_at_end=True: Specifies whether to load the best model at the end of training. If set to True, the best model will be loaded; if set to False, the last model will be loaded.

num_train_epochs=3: Specifies the number of epochs for training the model. In this case, the model will be trained for 3 epochs.

per_device_train_batch_size=2: Specifies the batch size for training. In this case, each training batch will contain 2 examples.

per_device_eval_batch_size=2: Specifies the batch size for evaluation. In this case, each evaluation batch will contain 2 examples.

In [24]:

'''
AutoModelForSequenceClassification is a class in the Transformers library that is used for sequence classification tasks, 
where the input is a sequence of text and the output is a label or category assigned to that sequence.

The benefit of using AutoModelForSequenceClassification is that it automatically selects the 
appropriate pre-trained model architecture based on the specified configuration and dataset. 
This makes it easy to fine-tune pre-trained models for various sequence classification tasks without having 
to manually select the appropriate model architecture.
'''

# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)
model_roberta = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=3) 
model_distilbert = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3) 

'''
Sentiment analysis is a common use case for sequence classification, 
where the goal is to classify text into categories such as positive, negative, or neutral sentiment. 
Therefore, AutoModelForSequenceClassification is a suitable choice for building a sentiment analysis model using BERT.
'''


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

'\nSentiment analysis is a common use case for sequence classification, \nwhere the goal is to classify text into categories such as positive, negative, or neutral sentiment. \nTherefore, AutoModelForSequenceClassification is a suitable choice for building a sentiment analysis model using BERT.\n'

In [25]:
train_dataset_base = dataset_base['train'].shuffle(seed=10) #.select(range(40000)) # to select a part

train_dataset_roberta = dataset_roberta['train'].shuffle(seed=10)

train_dataset_distilbert = dataset_distilbert['train'].shuffle(seed=10)

'''
train_dataset is created by selecting the 'train' subset of the original dataset and 
shuffling it randomly using the shuffle() function with a specified seed value of 10. 
This ensures that the data samples are presented to the model in a randomized order during training.

'''

eval_dataset_base = dataset_base['eval'].shuffle(seed=10)

eval_dataset_roberta = dataset_roberta['eval'].shuffle(seed=10)

eval_dataset_distilbert = dataset_distilbert['eval'].shuffle(seed=10)

## other way to split the train set ... in the range you must use: 
# # int(num_rows*.8 ) for [0 - 80%] and  int(num_rows*.8 ),num_rows for the 20% ([80 - 100%])
# train_dataset = dataset['train'].shuffle(seed=10).select(range(40000))
# eval_dataset = dataset['train'].shuffle(seed=10).select(range(40000, 41000))

In [26]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    rmse = np.sqrt(np.mean((predictions - labels)**2))
    return {"rmse": rmse}


In [28]:
# Launch the learning process: training 

'''
trainer.train() launches the training process on the specified train_dataset.
'''

trainer_base.train()

'''

During training, the model's parameters will be updated to minimize the loss between the predicted outputs and the actual outputs. The process consists of forward and backward passes through the neural network, followed by parameter updates using an optimization algorithm (in this case, AdamW).

The trainer object will keep track of the training progress, 
including the current epoch, the number of steps completed, 
the average training loss, and the average evaluation loss 
(if an evaluation dataset is provided). 
The training will continue for the specified number of epochs (num_train_epochs in training_args) 
or until the stopping criterion is met (e.g., early stopping based on the evaluation loss).

'''

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


"\n\nDuring training, the model's parameters will be updated to minimize the loss between the predicted outputs and the actual outputs. The process consists of forward and backward passes through the neural network, followed by parameter updates using an optimization algorithm (in this case, AdamW).\n\nThe trainer object will keep track of the training progress, \nincluding the current epoch, the number of steps completed, \nthe average training loss, and the average evaluation loss \n(if an evaluation dataset is provided). \nThe training will continue for the specified number of epochs (num_train_epochs in training_args) \nor until the stopping criterion is met (e.g., early stopping based on the evaluation loss).\n\n"

In [29]:
trainer_roberta.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=250, training_loss=0.8054835815429687, metrics={'train_runtime': 360.7491, 'train_samples_per_second': 22.176, 'train_steps_per_second': 0.693, 'total_flos': 2104907341824000.0, 'train_loss': 0.8054835815429687, 'epoch': 1.0})

In [30]:
trainer_distilbert.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=250, training_loss=0.8311105346679688, metrics={'train_runtime': 185.3952, 'train_samples_per_second': 43.151, 'train_steps_per_second': 1.348, 'total_flos': 1059758088192000.0, 'train_loss': 0.8311105346679688, 'epoch': 1.0})

In [31]:

# Evaluate the models
eval_base = trainer_base.evaluate()
eval_roberta = trainer_roberta.evaluate()
eval_distilbert = trainer_distilbert.evaluate()

# Create a dictionary of the evaluation results
results_dict = {
    "base": eval_base,
    "roberta": eval_roberta,
    "distilbert": eval_distilbert
}

# Create a pandas DataFrame from the dictionary


# Create a list of dictionaries from the evaluation results
results_list = []

for model_name, eval_results in results_dict.items():
    results_list.append({
        "Model": model_name,
        "Loss": eval_results["eval_loss"],
        "RMSE": eval_results["eval_rmse"],
        "Runtime": eval_results["eval_runtime"],
        "Samples Per Second": eval_results["eval_samples_per_second"],
        "Steps Per Second": eval_results["eval_steps_per_second"],
        "Epoch": eval_results["epoch"]
    })

# Create a pandas DataFrame from the list of dictionaries
results_df = pd.DataFrame(results_list)

# Print the results
print(results_df)

        Model      Loss      RMSE  Runtime  Samples Per Second  \
0        base  0.674963  0.756778  27.4571              72.877   
1     roberta  0.596201  0.672885  26.3948              75.810   
2  distilbert  0.645511  0.732279  13.2765             150.718   

   Steps Per Second  Epoch  
0            24.292    1.0  
1            25.270    1.0  
2            50.239    1.0  


In [32]:
# Sort the results by "eval_rmse" in ascending order and get the name and state dict of the best model
best_model = results_df.loc[results_df['RMSE'].idxmin()]

print(best_model)


Model                  roberta
Loss                  0.596201
RMSE                  0.672885
Runtime                26.3948
Samples Per Second       75.81
Steps Per Second         25.27
Epoch                      1.0
Name: 1, dtype: object


In [33]:
# Find the model with the lowest RMSE
best_model_name = results_df['Model'][results_df['RMSE'].idxmin()]
best_model_name

'roberta'

**Note that you should only push the best model to the Hugging Face Model Hub if you are satisfied with its performance.**

---

❌ ❌ ❌ ❌ ❌ ❌

In [34]:
# Push the best model to the Hugging Face Model Hub
if best_model_name == 'base':
    trainer_base.push_to_hub(push_to_hub_model_id="best-model-base")
elif best_model_name == 'roberta':
    trainer_roberta.push_to_hub(push_to_hub_model_id="best-model-roberta")
elif best_model_name == 'distilbert':
    trainer_distilbert.push_to_hub(push_to_hub_model_id="best-model-distilbert")


Upload file pytorch_model.bin:   0%|          | 1.00/476M [00:00<?, ?B/s]

Upload file runs/Apr24_05-28-35_01235d280086/1682314648.065379/events.out.tfevents.1682314648.01235d280086.204…

Upload file runs/Apr24_05-28-35_01235d280086/events.out.tfevents.1682314269.01235d280086.2040.0:   0%|        …

Upload file runs/Apr24_05-28-35_01235d280086/events.out.tfevents.1682314648.01235d280086.2040.2:   0%|        …

Upload file runs/Apr24_05-28-35_01235d280086/1682315024.121349/events.out.tfevents.1682315024.01235d280086.204…

...

Upload file runs/Apr24_05-28-35_01235d280086/events.out.tfevents.1682315641.01235d280086.2040.7:   0%|        …

Upload file runs/Apr24_05-28-35_01235d280086/events.out.tfevents.1682315613.01235d280086.2040.6:   0%|        …

Upload file runs/Apr24_05-28-35_01235d280086/events.out.tfevents.1682315655.01235d280086.2040.8:   0%|        …

To https://huggingface.co/Abubakari/finetuned-Sentiment-classfication-model
   66ca02e..226c8cb  main -> main

   66ca02e..226c8cb  main -> main

To https://huggingface.co/Abubakari/finetuned-Sentiment-classfication-model
   226c8cb..d7e310a  main -> main

   226c8cb..d7e310a  main -> main



'https://huggingface.co/Abubakari/finetuned-Sentiment-classfication-model/commit/226c8cb5eec1eb54b732ffcd6cd23f94f9180bca'

Some checkpoints of the model are automatically saved locally in `test_trainer/` during the training.

You may also upload the model on the Hugging Face Platform... [Read more](https://huggingface.co/docs/hub/models-uploading)

This notebook is inspired by an article: [Fine-Tuning Bert for Tweets Classification ft. Hugging Face](https://medium.com/mlearning-ai/fine-tuning-bert-for-tweets-classification-ft-hugging-face-8afebadd5dbf)

Do not hesitaite to read more and to ask questions, the Learning is a lifelong activity.

distilbert-base-uncased is a variant of the BERT model, which is a transformer-based neural network architecture designed for natural language processing (NLP) tasks such as text classification, named entity recognition, question answering, and more.

The "uncased" in the model name refers to the fact that the model was trained on lowercased text, meaning that the tokenizer will convert all text to lowercase before encoding it. The "base" refers to the size of the model, which has approximately 66 million parameters.

The main difference between distilbert-base-uncased and the original BERT model is that distilbert-base-uncased has been "distilled" or compressed to be smaller and faster, while maintaining a similar level of performance to the original BERT model. This was achieved by removing some of the model's layers and reducing the hidden size of the model.

distilbert-base-uncased was trained on a large corpus of English text using a masked language modeling (MLM) objective. In MLM, some of the input tokens are masked, and the model is trained to predict the masked tokens based on the surrounding context. This helps the model learn contextual representations of words, which are useful for many NLP tasks.

The distilbert-base-uncased model can be fine-tuned on a variety of NLP tasks by adding a task-specific output layer on top of the pre-trained model and fine-tuning the entire network on task-specific data. This fine-tuning process allows the model to adapt to a particular task and achieve state-of-the-art performance on many benchmarks.